In [345]:
import pandas as pd
import numpy as np
import csv

In [346]:
for y in ["1996-97","1997-98","1998-99","1999-00","2000-01","2001-02","2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14","2014-15","2015-16","2016-17","2017-18"]:
    
    data = pd.read_csv(str("../ALL BASKETBALL/drive-data/csv/raw_shot_data/raw_shot_data_"+y+".csv"))

    data1 = data[["gameId","playerName","actionType", "shotDistance","shotMadeFlag"]]
    plays_dict = {}

    plays = set(data1["actionType"].values.tolist())
    for play in plays:

        if ("Hook" in play) | ("Turnaround" in play):
            plays_dict[play]="Post"
        elif ("Tip" in play) | ("Putback" in play) | ("Alley" in play):
            plays_dict[play]="Reb/Oop"
        elif "Driving" in play:
            plays_dict[play]="Drive"
        elif "Cutting" in play:
            plays_dict[play]="Cut"
        elif "Pull" in play:
            plays_dict[play]="Pullup"
        elif "Layup" in play:
            plays_dict[play]="Layup"
        elif "Dunk" in play:
            plays_dict[play]="Dunk"
        elif "Jump" in play:
            plays_dict[play]="Jumper"
        else:
            plays_dict[play]="invalid"
        
    data1["ShotClass"] = data1["actionType"].map(plays_dict)
    data1["ShotRange"] = np.where(data1["shotDistance"]>46, "47-94 Backcourt", 
                                 np.where(data1["shotDistance"]>34, "35-46 Near Half", 
                                         np.where(data1["shotDistance"]>27, "28-34 Deep", 
                                                  np.where(data1["shotDistance"]>23, "24-27 Three", 
                                                          np.where(data1["shotDistance"]>17, "18-23 Long Jumper", 
                                                                  np.where(data1["shotDistance"]>12, "13-17 Mid Range", 
                                                                          np.where(data1["shotDistance"]>8, "09-12 Short Range", 
                                                                                  np.where(data1["shotDistance"]>2, "03-08 Near Rim", 
                                                                                          "00-02 At Rim"))))))))
    shots = data1[["gameId","playerName","shotMadeFlag","ShotRange","ShotClass"]]
    shots.to_csv(str("shot_data/all_shots_"+y+".csv"),index=False,encoding="utf-8")
    shot_summ_df = shots.groupby(["playerName","ShotRange","ShotClass","shotMadeFlag"]).size().reset_index()
    shot_summ_df.columns=["playername","ShotRange","ShotClass","Outcome","FGA"]
    misses = shot_summ_df.loc[shot_summ_df["Outcome"]==0]
    misses = misses.rename(columns={"FGA":"Missed"}).reset_index(drop=True)
    makes = shot_summ_df.loc[shot_summ_df["Outcome"]==1]
    makes = makes.rename(columns={"FGA":"Made"}).reset_index(drop=True)
    shot_log = pd.merge(makes,misses,how="outer",on=["playername","ShotRange","ShotClass"])
    shot_log = shot_log.fillna(0).drop(columns=["Outcome_x","Outcome_y"])
    shot_log[["Made","Missed"]] = shot_log[["Made","Missed"]].astype(int)
    shot_log["ShotPts"] = np.where(shot_log["ShotRange"].isin(["47-94 Backcourt","35-46 Near Half","28-34 Deep","24-27 Three"]),3,2)
    shot_log["FGA"]=shot_log["Made"] + shot_log["Missed"]
    shot_log["PPS"]=shot_log["ShotPts"] * (shot_log["Made"]/shot_log["FGA"])
    shot_log["Points"]=shot_log["Made"] * shot_log["ShotPts"]
    shot_log = shot_log.sort_values(["playername","ShotRange","ShotClass"])
    shot_log = shot_log.drop(columns=["Made","Missed","FGA","ShotPts"]).reset_index(drop=True)
    shot_log.to_csv(str("shot_data/shot_log_"+y+".csv"),index=False,encoding="utf-8")
    shot_pivot = pd.DataFrame(shot_log.pivot_table(values=["PPS","Points"],index=["playername"],columns=["ShotClass","ShotRange"])).fillna(0)
    shot_pivot.columns = ['_'.join(col).rstrip() for col in shot_pivot.columns.values]
    shot_pivot.to_csv(str("shot_charts/shot_chart"+y+".csv"),encoding="utf-8")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [385]:
for y in ["1996-97","1997-98","1998-99","1999-00","2000-01","2001-02","2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14","2014-15","2015-16","2016-17","2017-18"]:
    if y=="1996-97":
        dfdf = pd.read_csv(str("shot_charts/shot_chart"+y+".csv"))
        dfdf["Year"] = y[:2]+y[-2:]
    elif y=="1999-00":
        tempdf = pd.read_csv(str("shot_charts/shot_chart"+y+".csv"))
        tempdf["Year"] = "2000"
        dfdf = pd.concat([dfdf,tempdf])        
    else:
        tempdf = pd.read_csv(str("shot_charts/shot_chart"+y+".csv"))
        tempdf["Year"] = y[:2]+y[-2:]
        dfdf = pd.concat([dfdf,tempdf])

ddd=dfdf.set_index(["playername","Year"]).fillna(0)
ddd.to_csv("shot_charts/all_shots.csv",encoding="utf-8")

In [419]:
past = pd.read_csv("data/past_seasons_totals.csv").drop(columns=["Unnamed: 0","Unnamed: 0.1"])
sd = ddd.reset_index()
sd["Year"]=sd["Year"].astype(float)
past["Player"] = past["Player"].str.replace(".","").str.replace("*","").str.replace(r"'\'","^")
past["Player"] = past["Player"].str.split('^').str[0]
past

,Year,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,formatname
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,0.368,NaN,...,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0,CurlyArmstrong
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,0.435,NaN,...,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0,CliffBarker
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,0.394,NaN,...,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0,LeoBarnhorst
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,0.312,NaN,...,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0,EdBartels
4,1950.0,Ralph Beard,G,22.0,INO,60.0,NaN,NaN,0.422,NaN,...,NaN,NaN,NaN,233.0,NaN,NaN,NaN,132.0,895.0,RalphBeard
5,1950.0,Gene Berce,G-F,23.0,TRI,3.0,NaN,NaN,0.275,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.0,10.0,GeneBerce
6,1950.0,Charlie Black,F-C,28.0,TOT,65.0,NaN,NaN,0.346,NaN,...,NaN,NaN,NaN,163.0,NaN,NaN,NaN,273.0,661.0,CharlieBlack
7,1950.0,Nelson Bobb,PG,25.0,PHW,57.0,NaN,NaN,0.396,NaN,...,NaN,NaN,NaN,46.0,NaN,NaN,NaN,97.0,242.0,NelsonBobb
8,1950.0,Jake Bornheimer,F-C,22.0,PHW,60.0,NaN,NaN,0.356,NaN,...,NaN,NaN,NaN,40.0,NaN,NaN,NaN,111.0,254.0,JakeBornheimer
9,1950.0,Vince Boryla,SF,22.0,NYK,59.0,NaN,NaN,0.426,NaN,...,NaN,NaN,NaN,95.0,NaN,NaN,NaN,203.0,612.0,VinceBoryla


In [399]:
join = pd.merge(past,sd,how="outer",left_on=["Player","Year"],right_on=["playername","Year"])
join.to_csv("test.csv")